#### Obtener Tickets Prefinalizados, este estado se presenta cuando la solución depende de un tercero

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import tkinter as tk
from tkcalendar import Calendar

load_dotenv()

%run conexionDB.ipynb

# Obtener la conexión
engine = obtener_conexion()

In [2]:
from datetime import datetime

# Fecha de inicio y salida a producción de la aplicación de soporte osTicket: 2 de febrero de 2019
fechaInicio = '2019-02-01'

# Fecha de fin: fecha actual (hoy)
fechaFin = datetime.now().strftime('%Y-%m-%d')

queryPrefinalizados = """
SELECT inf.number AS 'No. de Ticket', dep.id as "dept_id", DATE_ADD(inf.created, INTERVAL -6 HOUR) AS 'Fecha de creación', asu.subject AS 'Asunto', cue.poster AS 'Usuario', cat.topic AS 'Tema de ayuda', DATE_ADD(age.timestamp, INTERVAL -6 HOUR) AS 'Fecha de cierre', DATE_ADD(inf.lastupdate, INTERVAL -6 HOUR) AS 'Ultima actualización'
FROM ost_ticket AS inf 
LEFT JOIN ost_thread_entry AS cue ON inf.created=cue.created 
LEFT JOIN ost_ticket__cdata AS asu ON inf.ticket_id=asu.ticket_id 
LEFT JOIN ost_help_topic AS cat ON inf.topic_id=cat.topic_id 
LEFT JOIN ost_thread_event AS age ON cue.thread_id=age.thread_id
LEFT JOIN ost_department AS dep ON inf.dept_id = dep.id
WHERE age.staff_id > 0 AND age.state = 'closed' AND age.data = '{{"status":[2,"Pre-Finalizado"]}}' 
AND DATE_ADD(inf.created, INTERVAL -6 HOUR) BETWEEN '{0}' AND DATE_ADD('{1}', INTERVAL 1 DAY)
GROUP BY inf.number
""".format(fechaInicio, fechaFin)

# Imprimir la consulta para verificar
print(queryPrefinalizados)
    
# Ejecutar la consulta y almacenar los resultados en la variable global
df_prefinalizados = pd.read_sql(queryPrefinalizados, engine)
df_prefinalizados


SELECT inf.number AS 'No. de Ticket', dep.id as "dept_id", DATE_ADD(inf.created, INTERVAL -6 HOUR) AS 'Fecha de creación', asu.subject AS 'Asunto', cue.poster AS 'Usuario', cat.topic AS 'Tema de ayuda', DATE_ADD(age.timestamp, INTERVAL -6 HOUR) AS 'Fecha de cierre', DATE_ADD(inf.lastupdate, INTERVAL -6 HOUR) AS 'Ultima actualización'
FROM ost_ticket AS inf 
LEFT JOIN ost_thread_entry AS cue ON inf.created=cue.created 
LEFT JOIN ost_ticket__cdata AS asu ON inf.ticket_id=asu.ticket_id 
LEFT JOIN ost_help_topic AS cat ON inf.topic_id=cat.topic_id 
LEFT JOIN ost_thread_event AS age ON cue.thread_id=age.thread_id
LEFT JOIN ost_department AS dep ON inf.dept_id = dep.id
WHERE age.staff_id > 0 AND age.state = 'closed' AND age.data = '{"status":[2,"Pre-Finalizado"]}' 
AND DATE_ADD(inf.created, INTERVAL -6 HOUR) BETWEEN '2019-02-01' AND DATE_ADD('2025-01-25', INTERVAL 1 DAY)
GROUP BY inf.number



,No. de Ticket,dept_id,Fecha de creación,Asunto,Usuario,Tema de ayuda,Fecha de cierre,Ultima actualización
0,000006,3,2019-02-06 16:04:49,control biometrico rues,DIANA CORRAL HERNANDEZ,SII,2019-02-06 17:40:00,2019-02-06 17:40:00
1,000012,1,2019-02-07 07:21:24,pantalla con hormigueo,LEIDY CARDENAS GOMEZ,Otros,2019-02-07 10:27:25,2019-02-07 10:41:25
2,000013,2,2019-02-07 07:32:00,Cambio ubicación escritorio en modulo de trabajo,JORGE LUIS OVIEDO GUZMAN,Adecuaciones,2019-02-08 14:47:32,2019-02-08 14:47:32
3,000018,2,2019-02-07 10:02:30,Sillas y escritorio,DOLY VIVIANA ZEA SANDOVAL,Mantenimiento,2019-02-08 14:45:05,2019-02-08 14:45:05
4,000019,2,2019-02-07 10:44:09,Descansabrazos de la silla giratoria,NUBIA VALENCIA CARDONA,Mantenimiento,2019-02-08 14:41:09,2019-02-08 14:41:09
...,...,...,...,...,...,...,...,...
2757,041338,3,2025-01-23 09:51:11,RENOVACION RNT,JHOAN ARANGO SEGURA,RNT,2025-01-23 11:52:04,2025-01-23 11:52:04
2758,041340,3,2025-01-23 10:24:56,constitucion vue,NUDI ALEJANDRA MARQUEZ MORA,SII,2025-01-23 12:10:07,2025-01-23 16:37:33
2759,041345,3,2025-01-23 11:36:21,Certifica situación de control,XIMENA GONGORA PEREZ,SII,2025-01-23 16:37:44,2025-01-23 16:37:44
2760,041348,3,2025-01-23 11:49:42,Recuperación de sipref de Renovaciones,DORA MIREYA SANCHEZ OSPINA,SII,2025-01-23 18:08:06,2025-01-23 18:08:06


#### Unificar TI 1 y TI 3, ya que ambos son del mismo departamento o área

In [3]:
# Reemplazar los valores de la columna "id" donde el valor sea 3 por 1
df_prefinalizados.loc[df_prefinalizados["dept_id"] == 3, "dept_id"] = 1
df_prefinalizados

,No. de Ticket,dept_id,Fecha de creación,Asunto,Usuario,Tema de ayuda,Fecha de cierre,Ultima actualización
0,000006,1,2019-02-06 16:04:49,control biometrico rues,DIANA CORRAL HERNANDEZ,SII,2019-02-06 17:40:00,2019-02-06 17:40:00
1,000012,1,2019-02-07 07:21:24,pantalla con hormigueo,LEIDY CARDENAS GOMEZ,Otros,2019-02-07 10:27:25,2019-02-07 10:41:25
2,000013,2,2019-02-07 07:32:00,Cambio ubicación escritorio en modulo de trabajo,JORGE LUIS OVIEDO GUZMAN,Adecuaciones,2019-02-08 14:47:32,2019-02-08 14:47:32
3,000018,2,2019-02-07 10:02:30,Sillas y escritorio,DOLY VIVIANA ZEA SANDOVAL,Mantenimiento,2019-02-08 14:45:05,2019-02-08 14:45:05
4,000019,2,2019-02-07 10:44:09,Descansabrazos de la silla giratoria,NUBIA VALENCIA CARDONA,Mantenimiento,2019-02-08 14:41:09,2019-02-08 14:41:09
...,...,...,...,...,...,...,...,...
2757,041338,1,2025-01-23 09:51:11,RENOVACION RNT,JHOAN ARANGO SEGURA,RNT,2025-01-23 11:52:04,2025-01-23 11:52:04
2758,041340,1,2025-01-23 10:24:56,constitucion vue,NUDI ALEJANDRA MARQUEZ MORA,SII,2025-01-23 12:10:07,2025-01-23 16:37:33
2759,041345,1,2025-01-23 11:36:21,Certifica situación de control,XIMENA GONGORA PEREZ,SII,2025-01-23 16:37:44,2025-01-23 16:37:44
2760,041348,1,2025-01-23 11:49:42,Recuperación de sipref de Renovaciones,DORA MIREYA SANCHEZ OSPINA,SII,2025-01-23 18:08:06,2025-01-23 18:08:06
